In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 539]
right_bottom = [900, 539]
apex = [475, 300]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)



imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(400,320),(500,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)


rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)


image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)


#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)




imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)



rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)



image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)



#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)


In [ ]:
import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image
#img[0] = mpimg.imread('test_images/solidWhiteRight.jpg')
image = mpimg.imread('test_images/solidWhiteCurve.jpg')
#img[2] = mpimg.imread('test_images/solidYellowLeft.jpg')
#img[3] = mpimg.imread('test_images/solidYellowCurve.jpg')
#img[4] = mpimg.imread('test_images/solidYellowCurve2.jpg')
#img[5] = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)




imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)



rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)



image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)



#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)





In [ ]:
import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image
image = mpimg.imread('test_images/solidYellowLeft.jpg')
#img[3] = mpimg.imread('test_images/solidYellowCurve.jpg')
#img[4] = mpimg.imread('test_images/solidYellowCurve2.jpg')
#img[5] = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)




imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)



rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)



image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)



#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)

In [ ]:
import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image
image = mpimg.imread('test_images/solidYellowCurve.jpg')
#img[4] = mpimg.imread('test_images/solidYellowCurve2.jpg')
#img[5] = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)




imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)



rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)



image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)



#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)

In [ ]:
import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)



########################

#reading in an image

image = mpimg.imread('test_images/solidYellowCurve2.jpg')
#img[5] = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))




thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)




imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)



rho = 1
theta = np.pi/180
threshold = 15
min_line_len = 40
max_line_gap = 20
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)



image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)



#plt.imshow(color_select)
#plt.imshow(line_image)

plt.imshow(image2)

In [ ]:



image = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

#pull out x and y sizes and make copy of image
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)
line_image = np.copy(image)
image2 = np.copy(image)

red_threshold = 200
green_threshold = 200
blue_threshold = 200
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

left_bottom = [0, 535]
right_bottom = [900, 535]
apex = [525, 200]

fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Mask pixels below the threshold
color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                    (image[:,:,1] < rgb_threshold[1]) | \
                    (image[:,:,2] < rgb_threshold[2])

# Find the region inside the lines
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))

thresholds = (image[:,:,0] < rgb_threshold[0]) \
            |(image[:,:,1] < rgb_threshold[1])\
            |(image[:,:,2] < rgb_threshold[2])
color_select[thresholds] = [0,0,0]

# Find where image is both colored right and in the region
line_image[~color_thresholds & region_thresholds] = [255,0,0]
high_threshold = 150
low_threshold = 50
kernel_size = 3

image2 = grayscale(image)
image2 = gaussian_blur(image2, kernel_size)
image2 = canny(image2,low_threshold,high_threshold)

imshape = image2.shape
vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
image2 = region_of_interest(image2, vertices)

rho = 1
theta = np.pi/180
threshold = 30
min_line_len = 100
max_line_gap = 160
line_image = np.copy(image)*0 #creating a blank to draw lines on
image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)
image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)

plt.imshow(image2)
#return image2

In [ ]:
#import imageio
#imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    high_threshold = 150
    low_threshold = 50
    kernel_size = 3
    
    image2 = grayscale(image)
    image2 = gaussian_blur(image2, kernel_size)
    image2 = canny(image2,low_threshold,high_threshold)

    imshape = image2.shape
    vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
    image2 = region_of_interest(image2, vertices)

    rho = 1
    theta = np.pi/180
    threshold = 30
    min_line_len = 100
    max_line_gap = 160
    line_image = np.copy(image)*0 #creating a blank to draw lines on
    image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)
    image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)
    last = image2
    return last

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    high_threshold = 150
    low_threshold = 50
    kernel_size = 3
    
    image2 = grayscale(image)
    image2 = gaussian_blur(image2, kernel_size)
    image2 = canny(image2,low_threshold,high_threshold)

    imshape = image2.shape
    vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
    image2 = region_of_interest(image2, vertices)

    rho = 1
    theta = np.pi/180
    threshold = 30
    min_line_len = 100
    max_line_gap = 160
    line_image = np.copy(image)*0 #creating a blank to draw lines on
    image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)
    image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)
    last = image2
    return last

yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import os
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

########################
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
           #Define different arrays and initialize y_min and y_max
    rightSlope=[]
    rightX=[]
    rightY=[]
    leftSlope=[]
    leftX=[]
    leftY=[]
    yMin=320
    yMax=img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<yMin:
                yMin=y1
            #Calculate slope and assign it likewise
            slope= (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope
                rightSlope.append(slope)
                rightX.append(x1)
                rightX.append(x2)
                rightY.append(y1)
                rightY.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope
                leftSlope.append(slope)
                leftX.append(x1)
                leftX.append(x2)
                leftY.append(y1)
                leftY.append(y2)

    #Averaging the position of line segment right side
    avgRightSlope= sum(rightSlope)/float(len(rightSlope))
    avgRightX=sum(rightX)/float(len(rightX))
    avgRightY=sum(rightY)/float(len(rightY))
    #Calculate intercept for averaged right coordinates
    rightIntercept= avgRightY-(avgRightSlope*avgRightX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Right=(yMin-rightIntercept)/avgRightSlope
    x2Right=(yMax-rightIntercept)/avgRightSlope
    #Draw right line
    cv2.line(img, (int(x1Right), yMin), (int(x2Right), yMax), color, thickness)
    
    #Averaging the position of line segment left side
    avgLeftSlope= sum(leftSlope)/float(len(leftSlope))
    avgLeftX=sum(leftX)/float(len(leftX))
    avgLeftY=sum(leftY)/float(len(leftY))
    #Calculate intercept for averaged left coordinates
    leftIntercept= avgLeftY-(avgLeftSlope*avgLeftX)
    #Find x1,x2 for highest point, we already have y1,y2
    x1Left=(yMin-leftIntercept)/avgLeftSlope
    x2Left=(yMax-leftIntercept)/avgLeftSlope
    #Draw left line
    cv2.line(img, (int(x1Left), yMin), (int(x2Left), yMax), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    high_threshold = 150
    low_threshold = 50
    kernel_size = 3
    
    image2 = grayscale(image)
    image2 = gaussian_blur(image2, kernel_size)
    image2 = canny(image2,low_threshold,high_threshold)

    imshape = image2.shape
    vertices = np.array([[(0,imshape[0]),(450,320),(600,320),(imshape[1],imshape[0])]], dtype =np.int32)
    image2 = region_of_interest(image2, vertices)

    rho = 1
    theta = np.pi/180
    threshold = 30
    min_line_len = 100
    max_line_gap = 160
    line_image = np.copy(image)*0 #creating a blank to draw lines on
    image2 = hough_lines(image2, rho, theta, threshold, min_line_len, max_line_gap)
    image2 = weighted_img(image2, image, α=0.8, β=1., λ=0.)
    last = image2
    return last

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))


The project had its difficulties luckily I was able to complete it.
If I could make my pipeline better I would make it so that it could handle the challenge video which includes curving lines. I'll continue to work on it in the mean time. I would also like to stabilize the lines in all videos. Im not sure if it's the car moving or the actual lines themselves but I would like to do what I can. My current algorithm as stated previously is likely to fail on curving lane lanes.

*****new Review********
first for my  gaussian_blur(img, kernel_size) I instantiated the kernel_size variable to 3 because the gaussian_blur for this particular domain needs to be blurred out to the right level so that the hough_lines can detect the lane lines easily, and this will also reduce noise. (my original value was 3 but wasnt instantiated) (this was also a suggestion from my reviewer)

Next for my canny function arguments threshold_low and threshold_high I instantiated them, I instantiated them to threshold_low = 50 and threshold_high = 150 as per my reviewers suggestions, which are good values for the canny edge detection, this enables my algorithm's canny function to detect the edges of the lane lines to each the hough_lines being drawn. my original values were( low = 50, high = 100)

finally, I made 3 small changes to the hough_lines function parameters, my original parameters for threshold = 15, min_line_len = 40, and the minimum bound of the length of the hough_line = 40. I changed to 30, 100, and 160 respectively. (another suggestion from my reviewer)